In [0]:
%sql
create catalog if not exists telecom_catalog_assign;
create database if not exists telecom_catalog_assign.landing_zone;
create volume if not exists telecom_catalog_assign.landing_zone.landing_vol;


In [0]:
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/")
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/")
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/")

#Hai
Explain the difference between (Just google and understand why we are going for volume concept for prod ready systems):
a. Volume vs DBFS/FileStore
b. Why production teams prefer Volumes for regulated data

In [0]:
base_path = "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/"

In [0]:
customer_csv = """
101,Arun,31,Chennai,PREPAID 
102,Meera,45,Bangalore,POSTPAID 
103,Irfan,29,Hyderabad,PREPAID 
104,Raj,52,Mumbai,POSTPAID 
105,,27,Delhi,PREPAID 
106,Sneha,abc,Pune,PREPAID
"""
dbutils.fs.put(f"{base_path}/customer/customer.csv", customer_csv, overwrite=True)
dbutils.fs.ls(f"{base_path}/customer/")


In [0]:
usage_tsv = """customer_id\tvoice_mins\tdata_mb\tsms_count
101\t320\t1500\t20 
102\t120\t4000\t5 
103\t540\t600\t52 
104\t45\t200\t2 105\t0\t0\t0
"""
dbutils.fs.put(f"{base_path}/usage/usage.csv", usage_tsv, overwrite=True)
dbutils.fs.ls(base_path/usage/)




In [0]:
dbutils.fs.ls(base_path + "/usage/")

In [0]:
tower_logs_region1 = """
event_id|customer_id|tower_id|signal_strength|timestamp 
5001|101|TWR01|-80|2025-01-10 10:21:54 
5004|104|TWR05|-75|2025-01-10 11:01:12
"""
dbutils.fs.put(f"{base_path}/tower/tower_logs_region1.csv", tower_logs_region1, overwrite=True)
dbutils.fs.ls(base_path + "/tower/")

In [0]:
base_path = "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/"
cust_csv_df1 = spark.read.csv(path=[base_path + "/customer/",base_path + "/tower/",base_path + "/usage/"],inferSchema=True,header=True,sep=",",pathGlobFilter="*.csv",recursiveFileLookup=True)
#cust_csv_df1.write.format("delta").save(f"{base_path}/customer/customer_csv_df1")


3. Directory Read Use Cases
Read all tower logs using: Path glob filter (example: *.csv) Multiple paths input Recursive lookup

Demonstrate these 3 reads separately: Using pathGlobFilter Using list of paths in spark.read.csv([path1, path2]) Using .option("recursiveFileLookup","true")

Compare the outputs and understand when each should be used.

In [0]:

# 3.1 Read all tower logs using: Path glob filter (example: *.csv) Multiple paths input Recursive lookup
spark.read.csv(path=["/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/"],header=True,inferSchema=True,sep="|",pathGlobFilter="*.csv",recursiveFileLookup=True).show()

#3.2 Demonstrate these 3 reads separately: Using pathGlobFilter Using list of paths in spark.read.csv([path1, path2]) Using .option("recursiveFileLookup","true")
spark.read.option("header","True").option("inferSchema","True").option("sep","|").option("pathGlobFilter","*.csv").option("recursiveFileLookup","True").format("CSV").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/").show()

     

4. Schema Inference, Header, and Separator
Try the Customer, Usage files with the option and options using read.csv and format function:
header=false, inferSchema=false
or
header=true, inferSchema=true
Write a note on What changed when we use header or inferSchema with true/false?
How schema inference handled “abc” in age?

In [0]:
%python

# Customer
custdf =spark.read.options(header="False",inferSchema="True",sep=",",pathGlobFilter="*.csv",recursiveFileLookup="True").csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/")
custdf.show()

# Usage
usagedf=  spark.read.options(header="True",inferSchema="True",sep="\t",pathGlobFilter="*.csv",recursiveFileLookup="True").csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/")
usagedf.show()


5. Column Renaming Usecases
Apply column names using string using toDF function for customer data
Apply column names and datatype using the schema function for usage data
Apply column names and datatype using the StructType with IntegerType, StringType, TimestampType and other classes for towers data

In [0]:
%python
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,TimestampType
#5.1
custdf.toDF("Id","Name","Age","Location","SimType").show()

#5.2
custom_schema = StructType([StructField("cust_id",IntegerType(),False),StructField("voi_mins",IntegerType(),True),StructField("data_mb",IntegerType(),True),StructField("sms_count",IntegerType(),True)])

custom_df1=spark.read.schema(custom_schema).options(header="True", sep="\t").csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/")
custom_df1.show()

custom_df1.printSchema()

#5.3
tower_schema = StructType([
    StructField("eve_id", IntegerType(), True),
    StructField("customer_id", IntegerType(), True),
    StructField("tower_id", StringType(), True),
    StructField("signal_strength", IntegerType(), True),
    StructField("event_timestamp", TimestampType(), True)
])

tower_df1=spark.read.schema(tower_schema).options(header="True", sep="|" ,pathGlobFilter="*.csv",recursiveFileLookup="True").csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/")
tower_df1.show()

#6. Write Operations (Data Conversion/Schema migration) – CSV Format Usecases
Write customer data into CSV format using overwrite mode
Write usage data into CSV format using append mode
Write tower data into CSV format with header enabled and custom separator (|)
Read the tower data in a dataframe and show only 5 rows.
Download the file into local from the catalog volume location and see the data of any of the above files opening in a notepad++.

In [0]:
# Reading the CSV file and storing it in a datframe 

from pyspark.sql.types import StructType,StructField, StringType, IntegerType

custom_schema = StructType([
    StructField("customer_id", IntegerType(), True),
    StructField("customer_name", StringType(), True),
    StructField("customer_age", IntegerType(), True),
    StructField("customer_city", StringType(), True),
    StructField("customer_plan_type", StringType(), True)])

read_customer_df = spark.read.schema(custom_schema).csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv")

# 1.Write customer data into CSV format using overwrite mode
write_customer_csv_df = read_customer_df.write.options(header='true').mode("overwrite").csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/csvout/")
display(write_customer_csv_df)

# 2.Write usage data into CSV format using append mode
write_customer_csv_df = read_customer_df.write.options(header='true').mode("append").csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/csvout/")
display(write_customer_csv_df)

# 3.Write tower data into CSV format with header enabled and custom separator (|)
read_tower_df = spark.read.options(header='true',sep='|',inferSchema='true',pathGlobeFilter='.csv',recursiveFileLookup='true').format('csv').load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower*")
display(read_tower_df)

write_tower_csv_df = read_tower_df.write.options(header='true',sep='|').mode("overwrite").csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/csvout/")
display(write_tower_csv_df)

# 4.Read the tower data in a dataframe and show only 5 rows.
display(read_tower_df.limit(5))

# 5.Download the file into local from the catalog volume location and see the data of any of the above files opening in a notepad++.
'''Yes, I could download the file into local from the catalog volume location and see the data of above files opening in a notepad++.'''




#7. Write Operations (Data Conversion/Schema migration)– JSON Format Usecases
1 Write customer data into JSON format using overwrite mode<br>
2 Write usage data into JSON format using append mode and snappy compression format<br>
3 Write tower data into JSON format using ignore mode and observe the behavior of this mode<br>
4 Read the tower data in a dataframe and show only 5 rows.<br>
5.Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.

In [0]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

custom_schema = StructType([
    StructField("customer_id", IntegerType(), True),
    StructField("customer_name", StringType(), True),
    StructField("customer_age", IntegerType(), True),
    StructField("customer_city", StringType(), True),
    StructField("customer_plan_type", StringType(), True)])

read_cust_json_df = spark.read.schema(custom_schema).csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv")
wte_cust_json_df = read_cust_json_df.write.mode("overwrite").format("json").save("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/jsonout/")
display(wte_cust_json_df)

#read_usage_json_df = spark.read.schema(custom_schema).csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.csv")
read_usage_json_df = spark.read.options(header='true',inferSchema="True",sep ='\t').csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.csv")
display(read_usage_json_df)
write_usage_json_df = read_usage_json_df.write.mode("append").option("compression","snappy").format("json").save("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/jsonout/")
display(write_usage_json_df)

#3.Write tower data into JSON format using ignore mode and observe the behavior of this mode
read_tower_df = spark.read.options(header='true',sep='|',inferSchema='true',recursiveFileLookup='true',pathGlobFilter='*.csv').csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/")
display(read_tower_df.limit)
write_tower_json_df = read_tower_df.write.mode("ignore").json("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/jsonout/")

display(read_tower_df.limit(5))




#8. Write Operations (Data Conversion/Schema migration) – Parquet Format Usecases
1.Write customer data into Parquet format using overwrite mode and in a gzip format<br>
2.Write usage data into Parquet format using error mode<br>
3.Write tower data into Parquet format with gzip compression option<br>
4.Read the usage data in a dataframe and show only 5 rows.<br>
5.Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.

In [0]:

from pyspark.sql.types import StructType,StructField, StringType, IntegerType
cust_schem = StructType([
    StructField("customer_id", IntegerType(), True),
    StructField("customer_name", StringType(), True),
    StructField("customer_age", IntegerType(), True),
    StructField("customer_city", StringType(), True),
    StructField("customer_plan_type", StringType(), True)])

cust_csv_df1 = spark.read.schema(cust_schem).csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv")
cust_csv_df1.show()
#wrt_prqt_cust_df = cust_csv_df1.write.mode("overwrite").format("parquet").save("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/prqtout/")
wrt_prqt_cust_df = cust_csv_df1.write.mode("overwrite").option("compression","gzip").format("parquet").save("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/prqtout/")
display(wrt_prqt_cust_df)


In [0]:
#8.2
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
cust_schem = StructType([
    StructField("customer_id", IntegerType(), True),
    StructField("voice_mins", IntegerType(), True),
    StructField("data_mb", IntegerType(), True),
    StructField("sms_count", IntegerType(), True)])

usg_csv_df1 = spark.read.schema(cust_schem).csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.csv")
usg_csv_df1.show()
#wrt_prqt_cust_df = cust_csv_df1.write.mode("overwrite").format("parquet").save("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/prqtout/")
#wrt_prqt_cust_df = cust_csv_df1.write.mode("overwrite").option("compression","gzip").format("parquet").save("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/prqtout/")
#display(wrt_prqt_cust_df)
wrt_prqt_usg_df = usg_csv_df1.write.mode("errorIfExists").option("compression","gzip").format("parquet").save("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/prqtout/")
display(wrt_prqt_usg_df)

In [0]:
#8.3
from pyspark.sql.types import StructType,StructField, StringType
cust_schem = StructType([StructField("event_id",IntegerType(),True), StructField("customer_id", IntegerType(), True),StructField("tower_id", StringType(), True),StructField("signal_strength", IntegerType(), True),StructField("timestamp", StringType(), True)])

tower_csv_df1 = spark.read.schema(cust_schem).options(headder="false",inferSchema="true",sep="|") .csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region1.csv")
tower_csv_df1.show()  
wrt_prqt_twr_usg_df1 = tower_csv_df1.write.mode("overwrite").option("compression","gzip").format("parquet").save("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/prqtout/")
display(tower_csv_df1)
display(wrt_prqt_twr_usg_df1.limit(5))
display(tower_csv_df1.limit(5))

In [0]:
read_usage_data_df1 = spark.read.options(header='true',inferschema='true',sep=',').csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.csv")
display(read_usage_data_df1.show(5))

read_usage_data_df2 = spark.read.options(header='true',inferschema='true',sep=',').csv("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.csv")
display(read_usage_data_df2.show(5))


##9. Write Operations (Data Conversion/Schema migration) – Orc Format Usecases<br>
Write customer data into ORC format using overwrite mode<br>
Write usage data into ORC format using append mode<br>
Write tower data into ORC format and see the output file structure<br>
Read the usage data in a dataframe and show only 5 rows.<br>

In [0]:
#9.1
read_cust_orc_df1 = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv")
display(read_cust_orc_df1.show(5))

wrt_cust_orc_df1= read_cust_orc_df1.write.mode("overwrite").format("orc").save("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/orcout/")
display(wrt_cust_orc_df1.show(5))
   


In [0]:
#9.2
read_usage_df1 = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.csv")
display(read_usage_df1.show(5))
wrt_usage_df1 = read_usage_df1.write.mode("append").format("orc").save("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/orcout/")
#display(wrt_usage_df1.show(5))

In [0]:
#9.3
read_twr_df1 = spark.read.options(header='true',inferSchema='true',sep='|').csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region1.csv")
display(read_twr_df1.show(5))
wrt_twr_df1 = read_twr_df1.write.mode("overwrite").format("orc").save("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/orcout/")
display(spark.read.format('orc').load('dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/orcout/').show(5))


In [0]:
read_usage_data_df3 = spark.read.options(header='true',inferSchema='true',sep=',').csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.csv")
display(read_usage_data_df3.show(5))


##10. Write Operations (Data Conversion/Schema migration) – Delta Format Usecases<br>
Write customer data into Delta format using overwrite mode<br>
Write usage data into Delta format using append mode<br>
Write tower data into Delta format and see the output file structure<br>
Read the usage data in a dataframe and show only 5 rows.<br>
Compare the parquet location and delta location and try to understand what is the differentiating factor, as both are parquet files only.<br>


In [0]:
from pyspark.sql.types import StructType,StructField,IntegerType,StringType
cust_schema = StructType([StructField("cust_id",IntegerType(),True),StructField("cust_name",StringType(),True),StructField("cust_age",IntegerType(),True),StructField("location",StringType(),True),StructField("plan",StringType(),True)])
read_cust_schema_df1 = spark.read.schema(cust_schema).options(header='true',inferSchema='true',sep=',').csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv")
display(read_cust_schema_df1.show(5))
wrt_cust_schema_df1 = read_cust_schema_df1.write.mode("overwrite").format("delta").save("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/deltaout/")




In [0]:
from pyspark.sql.types import StructType,StructField,IntegerType,StringType
usg_schema = StructType([StructField("cust_id",IntegerType(),True),StructField("voice_mins",IntegerType(),True),StructField("data_mdb",IntegerType(),True),StructField("sms_count",IntegerType(),True)])
read_usg_schema_df1 = spark.read.schema(usg_schema).options(header='true',inferSchema='true',sep=',').csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.csv")
display(read_usg_schema_df1.show(5))
wrt_usg_schema_df1 = read_usg_schema_df1.write.mode("append").format("delta").save("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/deltaout/")

In [0]:
from pyspark.sql.types import StructType,StructField,IntegerType,StringType
cust_schema_twr = StructType([StructField("event_id",IntegerType(),True),StructField("customer_id",IntegerType(),True),StructField("tower_id",StringType(),True),StructField("signal_strength",IntegerType(),True),StructField("event_time",StringType(),True)])
read_cust_schema_twr_df = spark.read.schema(cust_schema_twr).options(header='true',inferSchema='true',sep='|').csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region1.csv")
display(read_cust_schema_twr_df.show(5))
wrt_cust_schema_twr_df = read_cust_schema_twr_df.write.mode("overwrite").format("delta").save("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/deltaout/")
display(wrt_cust_schema_twr_df.show(5))

read_usage_data_df1 = spark.read.options(header='true',inferSchema='true',sep='|').format("delta").load("dbfs:////Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/deltaout/")
display(read_usage_data_df1.show(5))



In [0]:
from pyspark.sql.types import StructType,StructField,IntegerType,StringType,DoubleType,TimestampType
cust_usage_schema = StructType([StructField("cust_id", IntegerType(), True),
                                  StructField("voice_mins",IntegerType(),True),
                                  StructField("data_mb",IntegerType(),True),
                                  StructField("sms_count",DoubleType(),True)])

read_usage_df1 = spark.read.format("csv").schema(cust_usage_schema).load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.csv", sep = ',',header = True)
display(read_usage_df1)

read_usage_df1.write.format("delta").mode("overwrite").save("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage_raw_target_delta")
display(read_usage_df1.show(5))


##11 Write Operations (Lakehouse Usecases) – Delta table Usecases<br>
Write customer data using saveAsTable() as a managed table<br>
Write usage data using saveAsTable() with overwrite mode<br>
Drop the managed table and verify data removal<br>
Go and check the table overview and realize it is in delta format in the Catalog.<br>
Use spark.read.sql to write some simple queries on the above tables created.<br>

In [0]:
#11.1
read_cust_df1 = spark.read.csv("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv").toDF("customer_id","customer_name","customer_age","customer_city","customer_plan_type")
display(read_cust_df1.show(5))
wrt_cust_df1 = read_cust_df1.write.mode("overwrite").format("delta").saveAsTable("telecom_catalog_assign.landing_zone.customer_table")
write_sql= spark.sql("select * from telecom_catalog_assign.landing_zone.customer_table")
display(write_sql)


In [0]:
read_usage_df1 = spark.read.format("delta").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage_raw_target_delta")
display(read_usage_df1.show(5))
wrt_usage_tbl_df1 = read_usage_df1.write.mode("overwrite").format("delta").saveAsTable("telecom_catalog_assign.landing_zone.usage_table")
write_sql= spark.sql("select * from telecom_catalog_assign.landing_zone.usage_table")
display(write_sql)
display(wrt_usage_tbl_df1)


In [0]:
%sql
drop table telecom_catalog_assign.landing_zone.customer_table

In [0]:

drp_vew_cust = spark.sql("select * from telecom_catalog_assign.landing_zone.customer_table")
df_cust1 = spark.sql( "select * from telecom_catalog_assign.landing_zone.customer_delta" )
display(df_cust1)
df_usage2 = spark.sql("select * from telecom_catalog_assign.landing_zone.usage_table")
display(df_usage2)

#12. Write Operations (Lakehouse Usecases) – Delta table Usecases<br>
Write customer data using insertInto() in a new table and find the behavior<br>
Write usage data using insertTable() with overwrite mode

In [0]:
#1. Write customer data using insertInto() in a new table and find the behavior
df_temp = spark.createDataFrame([(103, "Balaji",45,"Chennai","PREPAID"), (110, "Bala",20,"Madurai","POSTPAID"), (113, "vasanth",35,"Thirchy","PREPAID")],schema=["customer_id","customer_name","customer_age","customer_city","customer_plan_type"])

df_temp.write.insertInto("telecom_catalog_assign.landing_zone.customer_table")
drp_vew_cust = spark.sql("select * from telecom_catalog_assign.landing_zone.customer_table")
display(drp_vew_cust)

In [0]:
#2. Write usage data using insertTable() with overwrite mode
df_usage = spark.sql( "select * from telecom_catalog_assign.landing_zone.usage_table" )
display(df_usage)

df_usage1 = spark.createDataFrame([(106,450,1000,12),(107,239,8910,34)],schema=["cust_id","voice_mins","data_mb","sms_count"])
display(df_usage1)

df_usage1.write.insertInto("telecom_catalog_assign.landing_zone.usage_table")


#13. Write Operations (Lakehouse Usecases) – Delta table Usecases<br>
Write customer data into XML format using rowTag as cust<br>
Write usage data into XML format using overwrite mode with the rowTag as usage<br>
Download the xml data and open the file in notepad++ and see how the xml file looks like.

In [0]:
#1. Write customer data into XML format using rowTag as cust
cust_df1 = spark.sql( "select * from telecom_catalog_assign.landing_zone.customer_table" )
display(cust_df1)
cust_df1.write.format("xml").mode("overwrite").option("rowTag","customer").save("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer_xml")


In [0]:
#2. Write usage data into XML format using overwrite mode with the rowTag as usage
usage_df1 = spark.sql( "select * from telecom_catalog_assign.landing_zone.usage_table" )
display(usage_df1)
usage_df1.write.format("xml").mode("overwrite").option("rowTag","usage").save("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage_xml")

#14. Compare all the downloaded files (csv, json, orc, parquet, delta and xml)<br>
Capture the size occupied between all of these file formats and list the formats below based on the order of size from small to big.